In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional, Conv2D, Input
# from tensorflow.keras.optimizers import SGD
# from tensorflow.random import set_seed

# set_seed(455)
# np.random.seed(455)

In [2]:
df = pd.read_csv('predict-fetal-acidosis/train.csv')
df

,SampleId,Category,fT0000,fT0001,fT0002,fT0003,fT0004,fT0005,fT0006,fT0007,...,uT2390,uT2391,uT2392,uT2393,uT2394,uT2395,uT2396,uT2397,uT2398,uT2399
0,P000001,0,150.50,150.50,151.00,151.25,151.25,150.25,150.25,150.25,...,5.5,5.0,4.0,4.5,4.5,4.5,5.0,5.0,5.5,7.0
1,P000002,0,141.00,141.00,141.00,139.75,139.75,139.75,139.75,140.25,...,14.0,13.5,13.5,13.5,13.5,13.5,13.5,13.0,13.5,13.0
2,P000003,0,138.25,138.25,139.50,139.50,139.50,139.50,140.25,140.25,...,52.0,51.0,44.0,39.5,48.0,54.0,54.0,47.5,41.5,42.5
3,P000004,0,114.50,114.50,114.50,114.50,116.00,116.00,116.50,117.75,...,36.5,31.5,40.0,47.5,49.5,50.5,49.5,50.0,49.5,48.0
4,P000005,0,113.50,112.50,112.50,114.00,114.00,115.50,115.50,112.50,...,25.5,28.5,30.5,22.5,24.0,24.5,24.5,25.0,25.0,24.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,P002996,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,18.0,12.5,14.5,21.5,21.0,16.5,15.0,12.0,9.5,9.0
2996,P002997,0,119.25,119.25,119.25,119.25,119.25,118.75,118.75,118.75,...,34.5,31.0,26.0,26.5,31.5,34.5,38.0,37.5,34.0,32.0
2997,P002998,0,130.25,129.75,129.75,129.00,129.00,128.00,128.00,127.25,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,0.0
2998,P002999,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
df.isna().sum()

SampleId    0
Category    0
fT0000      0
fT0001      0
fT0002      0
           ..
uT2395      0
uT2396      0
uT2397      0
uT2398      0
uT2399      0
Length: 4802, dtype: int64

In [4]:
for col in df.columns:
    print(col)

SampleId
Category
fT0000
fT0001
fT0002
fT0003
fT0004
fT0005
fT0006
fT0007
fT0008
fT0009
fT0010
fT0011
fT0012
fT0013
fT0014
fT0015
fT0016
fT0017
fT0018
fT0019
fT0020
fT0021
fT0022
fT0023
fT0024
fT0025
fT0026
fT0027
fT0028
fT0029
fT0030
fT0031
fT0032
fT0033
fT0034
fT0035
fT0036
fT0037
fT0038
fT0039
fT0040
fT0041
fT0042
fT0043
fT0044
fT0045
fT0046
fT0047
fT0048
fT0049
fT0050
fT0051
fT0052
fT0053
fT0054
fT0055
fT0056
fT0057
fT0058
fT0059
fT0060
fT0061
fT0062
fT0063
fT0064
fT0065
fT0066
fT0067
fT0068
fT0069
fT0070
fT0071
fT0072
fT0073
fT0074
fT0075
fT0076
fT0077
fT0078
fT0079
fT0080
fT0081
fT0082
fT0083
fT0084
fT0085
fT0086
fT0087
fT0088
fT0089
fT0090
fT0091
fT0092
fT0093
fT0094
fT0095
fT0096
fT0097
fT0098
fT0099
fT0100
fT0101
fT0102
fT0103
fT0104
fT0105
fT0106
fT0107
fT0108
fT0109
fT0110
fT0111
fT0112
fT0113
fT0114
fT0115
fT0116
fT0117
fT0118
fT0119
fT0120
fT0121
fT0122
fT0123
fT0124
fT0125
fT0126
fT0127
fT0128
fT0129
fT0130
fT0131
fT0132
fT0133
fT0134
fT0135
fT0136
fT0137
fT0138
fT0139
fT

In [5]:
df = df.drop(['SampleId'], axis=1)
df

,Category,fT0000,fT0001,fT0002,fT0003,fT0004,fT0005,fT0006,fT0007,fT0008,...,uT2390,uT2391,uT2392,uT2393,uT2394,uT2395,uT2396,uT2397,uT2398,uT2399
0,0,150.50,150.50,151.00,151.25,151.25,150.25,150.25,150.25,148.75,...,5.5,5.0,4.0,4.5,4.5,4.5,5.0,5.0,5.5,7.0
1,0,141.00,141.00,141.00,139.75,139.75,139.75,139.75,140.25,139.75,...,14.0,13.5,13.5,13.5,13.5,13.5,13.5,13.0,13.5,13.0
2,0,138.25,138.25,139.50,139.50,139.50,139.50,140.25,140.25,140.50,...,52.0,51.0,44.0,39.5,48.0,54.0,54.0,47.5,41.5,42.5
3,0,114.50,114.50,114.50,114.50,116.00,116.00,116.50,117.75,117.75,...,36.5,31.5,40.0,47.5,49.5,50.5,49.5,50.0,49.5,48.0
4,0,113.50,112.50,112.50,114.00,114.00,115.50,115.50,112.50,112.50,...,25.5,28.5,30.5,22.5,24.0,24.5,24.5,25.0,25.0,24.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,18.0,12.5,14.5,21.5,21.0,16.5,15.0,12.0,9.5,9.0
2996,0,119.25,119.25,119.25,119.25,119.25,118.75,118.75,118.75,118.75,...,34.5,31.0,26.0,26.5,31.5,34.5,38.0,37.5,34.0,32.0
2997,0,130.25,129.75,129.75,129.00,129.00,128.00,128.00,127.25,127.00,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,0.0
2998,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df_0 = df[df['Category'] == 0]
df_0.reset_index(inplace=True)
df_0.set_index('index')
df_0.drop(['index'], axis=1, inplace=True)
df_0

C:\Users\212806690\AppData\Local\Temp\ipykernel_22444\3865330672.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_0.drop(['index'], axis=1, inplace=True)


,Category,fT0000,fT0001,fT0002,fT0003,fT0004,fT0005,fT0006,fT0007,fT0008,...,uT2390,uT2391,uT2392,uT2393,uT2394,uT2395,uT2396,uT2397,uT2398,uT2399
0,0,150.50,150.50,151.00,151.25,151.25,150.25,150.25,150.25,148.75,...,5.5,5.0,4.0,4.5,4.5,4.5,5.0,5.0,5.5,7.0
1,0,141.00,141.00,141.00,139.75,139.75,139.75,139.75,140.25,139.75,...,14.0,13.5,13.5,13.5,13.5,13.5,13.5,13.0,13.5,13.0
2,0,138.25,138.25,139.50,139.50,139.50,139.50,140.25,140.25,140.50,...,52.0,51.0,44.0,39.5,48.0,54.0,54.0,47.5,41.5,42.5
3,0,114.50,114.50,114.50,114.50,116.00,116.00,116.50,117.75,117.75,...,36.5,31.5,40.0,47.5,49.5,50.5,49.5,50.0,49.5,48.0
4,0,113.50,112.50,112.50,114.00,114.00,115.50,115.50,112.50,112.50,...,25.5,28.5,30.5,22.5,24.0,24.5,24.5,25.0,25.0,24.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2735,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,18.0,12.5,14.5,21.5,21.0,16.5,15.0,12.0,9.5,9.0
2736,0,119.25,119.25,119.25,119.25,119.25,118.75,118.75,118.75,118.75,...,34.5,31.0,26.0,26.5,31.5,34.5,38.0,37.5,34.0,32.0
2737,0,130.25,129.75,129.75,129.00,129.00,128.00,128.00,127.25,127.00,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,0.0
2738,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df_1 = df[df['Category'] == 1]
df_1.reset_index(inplace=True)
df_1.set_index('index')
df_1.drop(['index'], axis=1, inplace=True)
df_1

C:\Users\212806690\AppData\Local\Temp\ipykernel_22444\3849612325.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1.drop(['index'], axis=1, inplace=True)


,Category,fT0000,fT0001,fT0002,fT0003,fT0004,fT0005,fT0006,fT0007,fT0008,...,uT2390,uT2391,uT2392,uT2393,uT2394,uT2395,uT2396,uT2397,uT2398,uT2399
0,1,146.25,146.25,150.25,148.50,148.50,143.50,148.00,148.00,148.00,...,41.5,40.0,35.0,34.0,34.0,34.5,35.0,33.0,33.0,31.0
1,1,150.50,152.00,150.00,150.00,150.00,150.00,150.00,150.00,150.00,...,37.5,38.0,38.0,38.5,38.5,38.5,39.0,39.0,39.0,39.0
2,1,147.75,147.75,147.75,147.25,147.25,146.25,146.25,146.25,146.50,...,29.0,29.5,29.5,29.5,30.0,30.0,28.5,28.5,27.5,27.5
3,1,151.75,152.00,152.75,152.75,153.00,153.25,153.25,153.00,153.00,...,15.0,14.5,14.0,14.0,14.5,15.0,15.5,15.5,15.0,14.5
4,1,94.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,20.5,21.0,20.0,19.5,19.5,19.5,20.0,20.5,20.5,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,1,139.50,139.50,138.50,139.50,139.50,140.50,140.50,140.50,140.75,...,75.0,75.0,74.5,74.0,73.5,73.5,73.5,73.5,73.5,74.0
256,1,83.25,83.25,83.25,87.00,87.00,87.00,82.75,82.75,82.75,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
257,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
258,1,159.50,159.50,159.50,159.00,159.00,159.25,159.00,159.00,158.75,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [18]:
df_0 = df_0.iloc[:300]
df_0

,Category,fT0000,fT0001,fT0002,fT0003,fT0004,fT0005,fT0006,fT0007,fT0008,...,uT2390,uT2391,uT2392,uT2393,uT2394,uT2395,uT2396,uT2397,uT2398,uT2399
0,0,150.50,150.50,151.00,151.25,151.25,150.25,150.25,150.25,148.75,...,5.5,5.0,4.0,4.5,4.5,4.5,5.0,5.0,5.5,7.0
1,0,141.00,141.00,141.00,139.75,139.75,139.75,139.75,140.25,139.75,...,14.0,13.5,13.5,13.5,13.5,13.5,13.5,13.0,13.5,13.0
2,0,138.25,138.25,139.50,139.50,139.50,139.50,140.25,140.25,140.50,...,52.0,51.0,44.0,39.5,48.0,54.0,54.0,47.5,41.5,42.5
3,0,114.50,114.50,114.50,114.50,116.00,116.00,116.50,117.75,117.75,...,36.5,31.5,40.0,47.5,49.5,50.5,49.5,50.0,49.5,48.0
4,0,113.50,112.50,112.50,114.00,114.00,115.50,115.50,112.50,112.50,...,25.5,28.5,30.5,22.5,24.0,24.5,24.5,25.0,25.0,24.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0,161.75,161.75,161.75,161.75,161.75,161.75,161.75,162.00,162.00,...,10.5,11.0,12.0,13.0,13.0,11.5,11.0,11.0,10.5,10.5
296,0,128.75,126.50,126.50,129.25,129.25,132.50,134.25,134.25,141.00,...,19.0,18.5,20.0,21.5,23.0,26.5,25.0,24.0,23.5,23.0
297,0,0.00,0.00,0.00,0.00,0.00,136.75,136.75,137.25,137.25,...,24.0,24.0,24.5,25.0,25.5,26.0,29.0,31.0,30.0,30.5
298,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,33.0,35.0,36.0,37.0,37.5,37.5,37.5,36.0,35.0,36.0


In [24]:
df = pd.concat([df_0, df_1], axis= 0)
df

,Category,fT0000,fT0001,fT0002,fT0003,fT0004,fT0005,fT0006,fT0007,fT0008,...,uT2390,uT2391,uT2392,uT2393,uT2394,uT2395,uT2396,uT2397,uT2398,uT2399
0,0,150.50,150.50,151.00,151.25,151.25,150.25,150.25,150.25,148.75,...,5.5,5.0,4.0,4.5,4.5,4.5,5.0,5.0,5.5,7.0
1,0,141.00,141.00,141.00,139.75,139.75,139.75,139.75,140.25,139.75,...,14.0,13.5,13.5,13.5,13.5,13.5,13.5,13.0,13.5,13.0
2,0,138.25,138.25,139.50,139.50,139.50,139.50,140.25,140.25,140.50,...,52.0,51.0,44.0,39.5,48.0,54.0,54.0,47.5,41.5,42.5
3,0,114.50,114.50,114.50,114.50,116.00,116.00,116.50,117.75,117.75,...,36.5,31.5,40.0,47.5,49.5,50.5,49.5,50.0,49.5,48.0
4,0,113.50,112.50,112.50,114.00,114.00,115.50,115.50,112.50,112.50,...,25.5,28.5,30.5,22.5,24.0,24.5,24.5,25.0,25.0,24.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,1,139.50,139.50,138.50,139.50,139.50,140.50,140.50,140.50,140.75,...,75.0,75.0,74.5,74.0,73.5,73.5,73.5,73.5,73.5,74.0
256,1,83.25,83.25,83.25,87.00,87.00,87.00,82.75,82.75,82.75,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
257,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
258,1,159.50,159.50,159.50,159.00,159.00,159.25,159.00,159.00,158.75,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [28]:
df = df.sample(frac=1)

In [29]:
X_train = df.drop(['Category'], axis=1).to_numpy()
X_train

array([[131.25, 131.25, 129.75, ...,  36.5 ,  36.5 ,  36.  ],
       [ 78.  ,  78.  ,  77.  , ...,  68.  ,  68.  ,  67.5 ],
       [140.25, 140.25, 140.75, ...,  89.  ,  89.  ,  90.  ],
       ...,
       [100.25, 100.25, 100.25, ...,   0.  ,   0.  ,   0.  ],
       [115.5 , 112.75, 112.75, ...,   0.  ,   0.  ,   0.  ],
       [118.5 , 118.25, 118.25, ...,  94.5 ,  85.5 ,  78.5 ]])

In [30]:
y_train = df['Category'].to_numpy()
y_train

array([0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,

In [31]:
X_train.shape

(560, 4800)

In [32]:
sc = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = sc.fit_transform(X_train)

In [34]:
X_train = X_train_scaled.reshape((X_train.shape[0], 2, 2400))

In [35]:
y_train.shape

(560,)

In [36]:
X_train.shape

(560, 2, 2400)

In [37]:
y_train = y_train.reshape(-1,1)
y_train.shape

(560, 1)

In [15]:
# X_train = X_train[:,:,:,np.newaxis]

In [38]:
X_train.shape

(560, 2, 2400)

In [17]:
# def split_sequence(sequence,labels, n_steps):
#     X, y = list(), list()
#     for i in range(len(sequence)):
#         end_ix = i + n_steps
#         if end_ix > len(sequence) - 1:
#             break
#         seq_x = sequence[i:end_ix]
#         seq_y = sequence[end_ix]
#         X.append(seq_x)
#         y.append(seq_y)
#     return np.array(X), np.array(y)


# n_steps = 60
# features = 1
# # split into samples
# X_train, y_train = split_sequence(training_set_scaled,test_set, n_steps)

In [39]:
tf.keras.backend.clear_session()
model = Sequential()
model.add(LSTM(units=125, activation="tanh", input_shape=(2, 2400)))
model.add(Dense(units=1, activation='sigmoid'))

In [40]:
model.compile(optimizer="Adam", loss='binary_crossentropy', metrics = ['accuracy'])

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 125)               1263000   
                                                                 
 dense (Dense)               (None, 1)                 126       
                                                                 
Total params: 1,263,126
Trainable params: 1,263,126
Non-trainable params: 0
_________________________________________________________________


In [42]:
history = model.fit(X_train, y_train, epochs=50)

Epoch 1/50
18/18 [==============================] - 3s 16ms/step - loss: 0.6969 - accuracy: 0.5196
Epoch 2/50
18/18 [==============================] - 0s 16ms/step - loss: 0.6797 - accuracy: 0.5714
Epoch 3/50
18/18 [==============================] - 0s 16ms/step - loss: 0.6754 - accuracy: 0.5875
Epoch 4/50
18/18 [==============================] - 0s 17ms/step - loss: 0.6653 - accuracy: 0.6018
Epoch 5/50
18/18 [==============================] - 0s 18ms/step - loss: 0.6582 - accuracy: 0.6286
Epoch 6/50
18/18 [==============================] - 0s 17ms/step - loss: 0.6438 - accuracy: 0.6464
Epoch 7/50
18/18 [==============================] - 0s 17ms/step - loss: 0.6374 - accuracy: 0.6482
Epoch 8/50
18/18 [==============================] - 0s 17ms/step - loss: 0.6149 - accuracy: 0.6804
Epoch 9/50
18/18 [==============================] - 0s 18ms/step - loss: 0.5902 - accuracy: 0.6839
Epoch 10/50
18/18 [==============================] - 0s 16ms/step - loss: 0.5853 - accuracy: 0.6804
Epoch 11/

In [43]:
df = pd.read_csv('predict-fetal-acidosis/test.csv')
df

,SampleId,fT0000,fT0001,fT0002,fT0003,fT0004,fT0005,fT0006,fT0007,fT0008,...,uT2390,uT2391,uT2392,uT2393,uT2394,uT2395,uT2396,uT2397,uT2398,uT2399
0,P003001,133.00,133.00,133.00,133.00,133.00,133.00,133.00,133.00,133.00,...,19.5,20.0,20.0,20.0,20.5,20.0,19.5,19.0,19.0,19.0
1,P003002,134.50,134.50,135.00,135.00,135.00,135.00,135.00,135.75,135.75,...,25.0,26.5,25.5,25.5,23.5,20.0,24.0,26.5,25.0,24.5
2,P003003,135.00,135.00,136.75,138.50,138.50,135.50,140.50,140.50,144.75,...,40.5,41.5,40.0,38.0,37.0,36.5,36.0,35.5,35.5,35.5
3,P003004,106.50,106.50,106.25,106.25,106.50,106.50,112.75,114.50,114.50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,P003005,118.50,115.50,115.50,115.25,115.25,115.50,115.50,115.25,115.25,...,28.0,25.5,23.5,21.5,20.5,20.5,21.0,23.0,25.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,P003942,117.25,117.25,115.00,115.00,107.25,107.25,100.25,100.25,100.25,...,11.0,9.0,9.0,11.0,11.0,13.0,13.0,13.0,13.0,12.0
942,P003943,95.00,95.00,95.50,95.50,95.25,95.25,95.25,95.25,95.25,...,46.0,46.0,46.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0
943,P003944,120.50,119.50,119.50,122.50,122.50,122.50,122.50,120.50,120.50,...,32.0,33.0,33.0,33.0,33.0,32.0,32.0,32.0,32.0,32.0
944,P003945,122.00,122.00,122.00,122.00,124.00,124.00,122.50,122.50,125.50,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [46]:
output_ls = df['SampleId'].to_numpy()
output_ls

array(['P003001', 'P003002', 'P003003', 'P003004', 'P003005', 'P003006',
       'P003007', 'P003008', 'P003009', 'P003010', 'P003011', 'P003012',
       'P003013', 'P003014', 'P003015', 'P003016', 'P003017', 'P003018',
       'P003019', 'P003020', 'P003021', 'P003022', 'P003023', 'P003024',
       'P003025', 'P003026', 'P003027', 'P003028', 'P003029', 'P003030',
       'P003031', 'P003032', 'P003033', 'P003034', 'P003035', 'P003036',
       'P003037', 'P003038', 'P003039', 'P003040', 'P003041', 'P003042',
       'P003043', 'P003044', 'P003045', 'P003046', 'P003047', 'P003048',
       'P003049', 'P003050', 'P003051', 'P003052', 'P003053', 'P003054',
       'P003055', 'P003056', 'P003057', 'P003058', 'P003059', 'P003060',
       'P003061', 'P003062', 'P003063', 'P003064', 'P003065', 'P003066',
       'P003067', 'P003068', 'P003069', 'P003070', 'P003071', 'P003072',
       'P003073', 'P003074', 'P003075', 'P003076', 'P003077', 'P003078',
       'P003079', 'P003080', 'P003081', 'P003082', 

In [47]:
X_test = df.drop(['SampleId'], axis=1).to_numpy()
X_test

array([[133.  , 133.  , 133.  , ...,  19.  ,  19.  ,  19.  ],
       [134.5 , 134.5 , 135.  , ...,  26.5 ,  25.  ,  24.5 ],
       [135.  , 135.  , 136.75, ...,  35.5 ,  35.5 ,  35.5 ],
       ...,
       [120.5 , 119.5 , 119.5 , ...,  32.  ,  32.  ,  32.  ],
       [122.  , 122.  , 122.  , ...,   3.  ,   3.  ,   3.  ],
       [  0.  ,   0.  ,   0.  , ...,   0.  ,   0.  ,   0.  ]])

In [48]:
X_test.shape

(946, 4800)

In [49]:
X_test_scaled = sc.fit_transform(X_test)
X_test_scaled.shape

(946, 4800)

In [50]:
X_test = X_test.reshape((X_test.shape[0],2,2400))
X_test.shape

(946, 2, 2400)

In [51]:
y_pred = model.predict(X_test)

30/30 [==============================] - 1s 5ms/step


In [54]:
count = 0
for e in y_pred:
    if e[0] > 0.5:
        count += 1
#         print(e)

print(count)

594


In [55]:
final_ls = (y_pred > 0.5).astype(int)
final_ls

array([[1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
    

In [56]:
final_ls.shape

(946, 1)

In [57]:
output_ls

array(['P003001', 'P003002', 'P003003', 'P003004', 'P003005', 'P003006',
       'P003007', 'P003008', 'P003009', 'P003010', 'P003011', 'P003012',
       'P003013', 'P003014', 'P003015', 'P003016', 'P003017', 'P003018',
       'P003019', 'P003020', 'P003021', 'P003022', 'P003023', 'P003024',
       'P003025', 'P003026', 'P003027', 'P003028', 'P003029', 'P003030',
       'P003031', 'P003032', 'P003033', 'P003034', 'P003035', 'P003036',
       'P003037', 'P003038', 'P003039', 'P003040', 'P003041', 'P003042',
       'P003043', 'P003044', 'P003045', 'P003046', 'P003047', 'P003048',
       'P003049', 'P003050', 'P003051', 'P003052', 'P003053', 'P003054',
       'P003055', 'P003056', 'P003057', 'P003058', 'P003059', 'P003060',
       'P003061', 'P003062', 'P003063', 'P003064', 'P003065', 'P003066',
       'P003067', 'P003068', 'P003069', 'P003070', 'P003071', 'P003072',
       'P003073', 'P003074', 'P003075', 'P003076', 'P003077', 'P003078',
       'P003079', 'P003080', 'P003081', 'P003082', 

In [58]:
result = [[]]
for i in range(output_ls.shape[0]):
    temp_ls = []
    temp_ls.append(output_ls[i])
    temp_ls.append(final_ls[i][0])
    
    result.append(temp_ls)

In [59]:
result = result[1:]

In [60]:
result = np.array(result)
for e in result:
    if e[1] == '1':
        print(e)

['P003001' '1']
['P003002' '1']
['P003003' '1']
['P003004' '1']
['P003005' '1']
['P003009' '1']
['P003010' '1']
['P003013' '1']
['P003016' '1']
['P003017' '1']
['P003020' '1']
['P003021' '1']
['P003022' '1']
['P003024' '1']
['P003025' '1']
['P003026' '1']
['P003027' '1']
['P003029' '1']
['P003030' '1']
['P003031' '1']
['P003032' '1']
['P003035' '1']
['P003037' '1']
['P003039' '1']
['P003043' '1']
['P003044' '1']
['P003045' '1']
['P003046' '1']
['P003048' '1']
['P003049' '1']
['P003050' '1']
['P003055' '1']
['P003060' '1']
['P003062' '1']
['P003063' '1']
['P003064' '1']
['P003065' '1']
['P003066' '1']
['P003067' '1']
['P003068' '1']
['P003070' '1']
['P003071' '1']
['P003073' '1']
['P003075' '1']
['P003077' '1']
['P003080' '1']
['P003081' '1']
['P003082' '1']
['P003083' '1']
['P003084' '1']
['P003085' '1']
['P003087' '1']
['P003088' '1']
['P003091' '1']
['P003092' '1']
['P003093' '1']
['P003095' '1']
['P003096' '1']
['P003097' '1']
['P003098' '1']
['P003099' '1']
['P003100' '1']
['P00310

In [61]:
type(result)

numpy.ndarray

In [62]:
df = pd.DataFrame(result, columns = ['SampleId','Category'])
df

,SampleId,Category
0,P003001,1
1,P003002,1
2,P003003,1
3,P003004,1
4,P003005,1
...,...,...
941,P003942,1
942,P003943,1
943,P003944,1
944,P003945,0


In [63]:
df[df['Category'] == '1']

,SampleId,Category
0,P003001,1
1,P003002,1
2,P003003,1
3,P003004,1
4,P003005,1
...,...,...
939,P003940,1
941,P003942,1
942,P003943,1
943,P003944,1


In [64]:
df.set_index('SampleId', inplace=True)
df

,Category
SampleId,
P003001,1
P003002,1
P003003,1
P003004,1
P003005,1
...,...
P003942,1
P003943,1
P003944,1


In [65]:
df[df['Category'] == '1']

,Category
SampleId,
P003001,1
P003002,1
P003003,1
P003004,1
P003005,1
...,...
P003940,1
P003942,1
P003943,1


In [66]:
df.to_csv('output.csv')